1. Importation des modules

In [80]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn import model_selection
from sklearn.model_selection import train_test_split


2. Extraction des données 

In [81]:
likes = pd.read_csv('Données_bêta_IzyLeaf/Données bêta IzyLeaf/like_0.csv')
users = pd.read_csv('Données_bêta_IzyLeaf/Données bêta IzyLeaf/user_0.csv')
voitures = pd.read_csv('Données_bêta_IzyLeaf/Données bêta IzyLeaf/car_0.csv')
like_2 = pd.read_csv('Données_bêta_IzyLeaf/Données bêta IzyLeaf/like_2.csv')
print(voitures)

       brand     model    dealer  \
0    CITROEN        C3  sofibrie   
1    RENAULT  Talisman  sofibrie   
2    RENAULT    Kadjar  sofibrie   
3    RENAULT    Koleos  sofibrie   
4    CITROEN        C3  sofibrie   
..       ...       ...       ...   
264     JEEP  Renegade   as_cars   
265     OPEL     Corsa   as_cars   
266  CITROEN        C3   as_cars   
267  RENAULT    Kadjar  sofibrie   
268   NISSAN   Qashqai   as_cars   

                                                  url  price  year      km  \
0    https://www.leboncoin.fr/voitures/2221069564.htm  12890  2017   60933   
1    https://www.leboncoin.fr/voitures/2218598351.htm  24900  2019   63510   
2    https://www.leboncoin.fr/voitures/2229908445.htm  20890  2019   58858   
3    https://www.leboncoin.fr/voitures/2301949775.htm  36490  2022      64   
4    https://www.leboncoin.fr/voitures/2274683789.htm  14890  2021   42555   
..                                                ...    ...   ...     ...   
264  https://www.lebo

Extraction des id des users ayant liké au moins une voiture lors du 2eme test de la bêta de IzyLeaf

In [32]:
user_indice2 = []
for u in list(set(list(like_2['user_id']))) :
    extract1 = len(like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== u),['car_id','value']])
    if extract1 != 0 :
        user_indice2.append(u)
    
print(user_indice2)


[980, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1005, 1006, 1007, 1009, 1010, 1011, 1013, 1014, 1019, 1020, 1021, 1022, 1023, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1033, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1113, 1114, 1115, 1116, 1118, 1119, 1120, 1121, 1122, 1123, 1125, 1126, 1127, 1128, 1130, 1131, 1132, 1133, 1134, 1136, 1137, 1138, 1140, 1142, 1143, 1145, 1146, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1157, 1158, 1159, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181,

recherche de l'utilisateur avec le plus de like pour tester le modèle 

In [33]:
nb_likes = 0
u_i = 0
for u in user_indice2 : 
    user_id = u
    extract1 = like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== user_id),['car_id','value']]
    if len(extract1) > nb_likes:
        nb_likes = len(extract1)
        u_i = u 

print(nb_likes)
print(u_i)

269
1151


fonction utile pour transformer les string en int

In [34]:

def trans_numb(string,l):
    a = l.index(string)
    return a

models = list(set(list(voitures['model'])))
print(models)


# Pour appliquer kNN il faut des données chiffrées, donc transfo marques en chiffres. Problèmes, pourquoi genre renault plus proche de Nissan que de mini ? 

['T-Cross', 'Mazda 5', 'Giulietta', 'Captur', 'CX-5', 'C3 Aircross', 'Crossland X', 'Matiz', 'Vitara', 'Countryman', 'Arkana', 'Clio', 'Range Rover', 'Grand C-MAX', 'T-Roc', 'Zoe', 'SpaceTourer', 'Explorer', 'Autre', 'Giulia', 'C3', 'DS 7', 'Tipo', 'Passat', 'Mazda 2', 'Jumpy Fg', 'X1', 'GS', 'Scenic', 'C4 Picasso', 'Talisman', 'Leon', '207', 'Focus', 'Tiguan', 'Swift', 'TT', 'Jazz', 'MiTo', 'Tucson', 'Golf', 'Touran', '308', 'Ibiza', '500', 'Karoq', 'XC60', 'Kangoo', 'Kadjar', 'Megane', '2008', 'Autres', '3008', 'Twingo', 'C-HR', 'Golf Sportsvan', 'Duster', 'Sandero', 'MX-5', 'Mokka', 'Micra', 'Arteon', 'Rifter', 'Trafic Combi', 'CX-30', '1007', 'Corsa', 'DS 4', 'Grand Cherokee', 'C1', 'Série 3', '5008', 'MX-3', 'Koleos', 'C4', 'DS 3', 'C5', 'Q5', 'Qashqai', 'A5', '208', 'Grandland X', 'Rio', 'Polo', 'Punto', 'Renegade', 'C-MAX', '508', 'A1', 'Mazda 3', 'Puma']


Sélection d'une numérotation pour les marques (attention 4min30 d'execution)

In [33]:
# faire genre 100 permutation de la liste, moyenne des score sur les users_actifs 
brands = ['ABARTH','FIAT','OPEL','RENAULT','CITROEN','DS','PEUGEOT','SKODA','FORD','TOYOTA','KIA','VOLKSWAGEN','DACIA','VOLVO','NISSAN','HYUNDAI','HONDA','SUZUKI','MAZDA','LEXUS','MINI','SEAT','AUDI','LAND-ROVER','JEEP','BMW','ALFA ROMEO','CHEVROLET']
mean_perm = [0,brands]
for perm in range(100):
    brand_perm = list(np.random.permutation(brands))
    scores = []
    for u in user_indice2 :
        extract = like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== u),['car_id','value']]
        X2 = pd.merge(extract,voitures)
        X2.dropna(inplace=True)
        y1 = X2['value']
        X2['brand'] = X2['brand'].apply(lambda x : trans_numb(x,brand_perm))
        X3 = X2['brand'].values.reshape(-1,1)
        if len(X3)>1:
            X_train1,X_test1,y_train1,y_test1 = train_test_split(X3,y1,test_size=0.2)
            if len(X_train1)>=4:
             model1 = KNeighborsRegressor(n_neighbors=4,weights='uniform')
             model1.fit(X_train1,y_train1)
             if len(y_test1)>1 :
                    note = model1.score(X_test1,y_test1)
                    scores.append(note)
    value = np.mean(scores)
    if value > mean_perm[0]:
        mean_perm[0]= value
        mean_perm[1]=brand_perm

print(mean_perm)
        
    

[0.8093320481222246, ['HONDA', 'SEAT', 'AUDI', 'TOYOTA', 'KIA', 'SUZUKI', 'HYUNDAI', 'CITROEN', 'NISSAN', 'FORD', 'RENAULT', 'PEUGEOT', 'LAND-ROVER', 'ALFA ROMEO', 'VOLKSWAGEN', 'ABARTH', 'VOLVO', 'MAZDA', 'SKODA', 'CHEVROLET', 'BMW', 'FIAT', 'DACIA', 'MINI', 'LEXUS', 'DS', 'OPEL', 'JEEP']]


Sélection d'une numérotation pour les modèles de voiture ( attention 4min30 d'execution)

In [34]:
models = list(set(list(voitures['model'])))
mean_perm1 = [0,models]
for perm in range(100):
    mod_perm = list(np.random.permutation(models))
    scores = []
    for u in user_indice2 :
        extract = like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== u),['car_id','value']]
        X2 = pd.merge(extract,voitures)
        X2.dropna(inplace=True)
        y1 = X2['value']
        X2['model'] = X2['model'].apply(lambda x : trans_numb(x,mod_perm))
        X3 = X2['model'].values.reshape(-1,1)
        if len(X3)>1:
            X_train1,X_test1,y_train1,y_test1 = train_test_split(X3,y1,test_size=0.2)
            if len(X_train1)>=4:
             model1 = KNeighborsRegressor(n_neighbors=4,weights='uniform')
             model1.fit(X_train1,y_train1)
             y_new = model1.predict(X_test1)
             if len(y_test1)>1 :
                    note = model1.score(X_test1,y_test1)
                    scores.append(note)
    value = np.mean(scores)
    if value > mean_perm1[0]:
        mean_perm1[0]= value
        mean_perm1[1]=mod_perm

print(mean_perm1)

[0.7859740363588258, ['SpaceTourer', 'Arteon', 'Zoe', 'XC60', 'C-HR', 'Kadjar', 'Autres', '3008', 'C3', 'Talisman', 'Jumpy Fg', 'Arkana', 'Grandland X', '1007', 'CX-30', 'Autre', 'Golf Sportsvan', '2008', 'Leon', 'Matiz', 'Mazda 3', 'MX-5', 'Q5', 'Countryman', 'TT', 'Megane', 'DS 3', 'Mokka', 'Punto', 'Twingo', 'Micra', 'Trafic Combi', 'Sandero', 'MiTo', 'Mazda 2', 'Tipo', 'C1', 'Scenic', 'Polo', 'C5', 'Explorer', 'CX-5', 'Renegade', 'Golf', 'Karoq', 'Swift', 'Focus', 'DS 7', 'Puma', 'Tucson', 'Clio', '500', 'Mazda 5', 'GS', 'Grand Cherokee', '308', '508', 'C-MAX', 'X1', 'Jazz', 'C4 Picasso', 'Giulietta', 'Qashqai', 'T-Roc', 'Grand C-MAX', '207', 'A5', 'C3 Aircross', 'Captur', 'Koleos', 'Vitara', 'T-Cross', '5008', 'A1', 'Giulia', 'DS 4', 'Série 3', 'Rifter', 'Rio', 'Kangoo', 'Duster', 'C4', '208', 'Crossland X', 'Passat', 'Tiguan', 'Corsa', 'Touran', 'Ibiza', 'Range Rover', 'MX-3']]


Listes servant à numéroter les variables de type string 

In [36]:
models = ['Arteon', 'Golf', 'Série 3', 'T-Roc', 'Zoe', 'Mazda 2', 'C-HR', 'Punto', 'A1', 'Mazda 3', 'C4', 'Puma', 'Arkana', '500', 'SpaceTourer', 'Tucson', 'Megane', 'Sandero', 'Focus', '5008', 'Karoq', 'Twingo', 'Leon', 'Golf Sportsvan', '508', 'Kadjar', 'Rio', 'Koleos', 'DS 7', 'Polo', 'Crossland X', 'Ibiza', 'Qashqai', 'C3', 'Grandland X', 'Clio', 'C5', '308', '1007', 'GS', 'Mazda 5', 'Passat', 'Vitara', 'A5', '207', 'Corsa', 'CX-5', 'MiTo', 'T-Cross', 'Mokka', 'Autre', 'Trafic Combi', 'DS 3', 'Grand Cherokee', '3008', 'Countryman', 'Matiz', 'Duster', 'C1', 'Talisman', 'TT', '2008', 'Q5', 'Autres', 'Kangoo', 'Captur', 'DS 4', 'C3 Aircross', 'Range Rover', 'C-MAX', 'MX-3', 'MX-5', 'Swift', 'C4 Picasso', 'XC60', 'Renegade', 'X1', '208', 'Grand C-MAX', 'Jumpy Fg', 'Jazz', 'Tiguan', 'CX-30', 'Rifter', 'Giulietta', 'Explorer', 'Touran', 'Micra', 'Scenic', 'Tipo', 'Giulia']
colors = ['Blanc','Gris','Noir','Vert','Bleu','Beige','Rouge','Marron','Argent','Jaune','Orange','Autre']
fuel = ['Diesel','Essence','Hybride','Electrique']
gearbox = ['Automatique','Manuelle']
brands = ['HONDA', 'LAND-ROVER', 'BMW', 'CHEVROLET', 'VOLKSWAGEN', 'CITROEN', 'MINI', 'HYUNDAI', 'MAZDA', 'DACIA', 'SEAT', 'FORD', 'LEXUS', 'AUDI', 'DS', 'ALFA ROMEO', 'FIAT', 'JEEP', 'PEUGEOT', 'SKODA', 'TOYOTA', 'NISSAN', 'OPEL', 'RENAULT', 'ABARTH', 'VOLVO', 'KIA', 'SUZUKI']




Transformation des données (string -> int)

In [75]:
voitures['brand'] = voitures['brand'].apply(lambda x : trans_numb(x,brands))
voitures['color'] = voitures['color'].apply(lambda x : trans_numb(x,colors))
voitures['model'] = voitures['model'].apply(lambda x : trans_numb(x,models))
voitures['gearbox'] = voitures['gearbox'].apply(lambda x : trans_numb(x,gearbox))
voitures['fuel'] = voitures['fuel'].apply(lambda x : trans_numb(x,fuel))


ValueError: 0.18518518518518517 is not in list

Normalisation des données 

In [74]:
transformer = preprocessing.MinMaxScaler().fit(voitures[['brand']])
voitures['brand']= transformer.transform(voitures[['brand']])

transformer = preprocessing.MinMaxScaler().fit(voitures[['color']])
voitures['color']= transformer.transform(voitures[['color']])

transformer = preprocessing.MinMaxScaler().fit(voitures[['model']])
voitures['model']= transformer.transform(voitures[['model']])

transformer = preprocessing.MinMaxScaler().fit(voitures[['gearbox']])
voitures['gearbox']= transformer.transform(voitures[['gearbox']])

transformer = preprocessing.MinMaxScaler().fit(voitures[['fuel']])
voitures['fuel']= transformer.transform(voitures[['fuel']])

transformer = preprocessing.MinMaxScaler().fit(voitures[['km']])
voitures['km']= transformer.transform(voitures[['km']])

transformer = preprocessing.MinMaxScaler().fit(voitures[['price']])
voitures['price']= transformer.transform(voitures[['price']])

voitures

,brand,model,dealer,url,price,year,km,fuel,gearbox,color,doors,seats,fiscalPower,DINPower,critAir,nb_img,link,car_id
0,0.185185,0.366667,sofibrie,https://www.leboncoin.fr/voitures/2221069564.htm,0.136923,2017,0.253856,0.333333,1.0,0.000000,5,5.0,4,82,1.0,7,NaN,2
1,0.851852,0.655556,sofibrie,https://www.leboncoin.fr/voitures/2218598351.htm,0.321692,2019,0.264594,0.333333,0.0,0.090909,5,5.0,8,159,1.0,20,NaN,3
2,0.851852,0.277778,sofibrie,https://www.leboncoin.fr/voitures/2229908445.htm,0.260000,2019,0.245210,0.000000,1.0,0.090909,5,5.0,6,116,2.0,20,NaN,4
3,0.851852,0.300000,sofibrie,https://www.leboncoin.fr/voitures/2301949775.htm,0.500000,2022,0.000225,0.333333,0.0,0.181818,5,5.0,8,158,1.0,20,NaN,5
4,0.185185,0.366667,sofibrie,https://www.leboncoin.fr/voitures/2274683789.htm,0.167692,2021,0.177278,0.333333,1.0,0.090909,5,5.0,4,83,1.0,20,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,0.629630,0.833333,as_cars,https://www.leboncoin.fr/voitures/2294835918.htm,0.184615,2016,0.547573,0.000000,1.0,0.181818,5,5.0,6,120,2.0,17,NaN,356
265,0.814815,0.500000,as_cars,https://www.leboncoin.fr/voitures/2233134235.htm,0.061538,2015,0.512480,0.000000,1.0,0.000000,5,5.0,4,75,2.0,14,NaN,357
266,0.185185,0.366667,as_cars,https://www.leboncoin.fr/voitures/2301027616.htm,0.038462,2011,0.477937,0.000000,1.0,0.545455,5,5.0,5,92,2.0,13,NaN,358
267,0.851852,0.277778,sofibrie,https://www.leboncoin.fr/voitures/2249176942.htm,0.250769,2018,0.305679,0.000000,1.0,0.090909,5,5.0,7,131,2.0,19,NaN,1


Permet de créer une liste d'index de set de train et test, pour avoir toujours les mêmes ensemble de train et teste dans la comparaison effectuée dans la case suivante. 

In [14]:
index_train_test= []
for u in user_indice2:
    extract = like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== u),['car_id','value']]
    X1 = pd.merge(extract,voitures)
    y = X1['value']
    X = X1.loc[:,['brand','model','price','km','fuel','gearbox','color']]
    long = len(X)
    if long>1 :
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
        ind_train = X_train.index.tolist()
        ind_test = X_test.index.tolist()
        index_train_test.append([ind_train,ind_test])
    else : 
        index_train_test.append([[],[]])


index_train_test

[[[2, 3, 0, 4, 1], [6, 5]],
 [[36,
   60,
   62,
   41,
   37,
   50,
   35,
   46,
   22,
   7,
   28,
   26,
   61,
   38,
   51,
   45,
   21,
   34,
   11,
   8,
   29,
   33,
   31,
   12,
   14,
   2,
   9,
   16,
   48,
   4,
   43,
   1,
   15,
   17,
   44,
   30,
   3,
   59,
   5,
   40,
   18,
   25,
   6,
   27,
   0,
   24,
   56,
   58,
   39,
   54],
  [57, 55, 10, 20, 23, 49, 13, 52, 42, 32, 47, 53, 19]],
 [[], []],
 [[10, 7, 0, 13, 8, 1, 17, 9, 19, 15, 12, 18, 14, 3, 6, 11], [5, 4, 16, 2]],
 [[8, 0, 1, 3, 6, 11, 7, 9, 2], [5, 4, 10]],
 [[12,
   4,
   31,
   8,
   11,
   33,
   23,
   29,
   24,
   16,
   35,
   38,
   3,
   32,
   0,
   14,
   28,
   25,
   9,
   19,
   15,
   2,
   5,
   30,
   27,
   7,
   39,
   10,
   17,
   20,
   21,
   26],
  [37, 18, 36, 22, 13, 1, 6, 34]],
 [[7,
   4,
   2,
   16,
   9,
   27,
   8,
   10,
   6,
   17,
   15,
   21,
   0,
   28,
   1,
   5,
   12,
   19,
   22,
   11,
   20,
   14,
   18],
  [3, 26, 24, 13, 25, 23]],
 [[1, 0,

Calcul du score moyen de kNN sur les différents ensembles de likes des users du test2 de la bêta d'IzyLeaf.

Le but de cette case était d'observer la variation du score lors du retrait de certaines variables, pour éliminer les variables qui faisaient baisser le score. Aucune variable de ce type n'a été trouvée, elles sont donc toutes conservées pour le modèle final. 

In [17]:

li_score = []
for u in range(len(user_indice2)):
    extract = like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== user_indice2[u]),['car_id','value']]
    X1 = pd.merge(extract,voitures)
    y = X1['value']
    X = X1.loc[:,['brand','model','km','price','fuel','gearbox','color']]
    long = len(X)
    if long>1 :
        X_train = X.loc[index_train_test[u][0]]
        X_test = X.loc[index_train_test[u][1]]
        y_train = y.loc[index_train_test[u][0]]
        y_test = y.loc[index_train_test[u][1]]
        if len(X_train)>=4:
            model = KNeighborsRegressor(n_neighbors=2,weights='uniform')
            model.fit(X_train,y_train)
            score = model.score(X_test,y_test)
            li_score.append(score)

print(np.mean(li_score))


    


c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\si

nan


c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [76]:
extract = like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== 1014),['car_id','value']]
X1 = pd.merge(extract,voitures)
print(len(extract))
y = X1['value']
X = X1.loc[:,['brand','model','km','price','fuel','gearbox','color']]

224


Choix du meilleur k 

In [77]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25]}


# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(
    KNeighborsRegressor(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=3,           # nombre de folds de validation croisée
     
)
# Optimiser ce classifieur sur le jeu d'entraînement
clf.fit(X_train, y_train)
nb_neighbors = clf.best_params_['n_neighbors']
print('nombre de voisins pris en compte :', nb_neighbors)

nombre de voisins pris en compte : 7


Entrainement du modèle 

In [78]:
model = KNeighborsRegressor(n_neighbors=nb_neighbors,weights='uniform')
model.fit(X_train,y_train)


KNeighborsRegressor(n_neighbors=7, weights='distance')

In [79]:
Y_NEW = model.predict(np.array(X_test))
print(model.score(np.array(X_test),np.array(y_test)))
print(Y_NEW)




0.15910467202753908
[-1.00000000e+00 -2.75609857e-01 -5.52076407e-01 -4.64327212e-01
 -3.43892620e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -7.97415952e-01 -1.00000000e+00 -7.57673966e-01 -1.00000000e+00
 -1.00000000e+00 -3.77267028e-01 -1.00000000e+00  3.71858611e-01
 -1.00000000e+00 -7.06610747e-01 -1.00000000e+00 -9.71690215e-01
 -1.00000000e+00 -7.98964323e-01 -1.00000000e+00 -7.89178606e-01
 -1.00000000e+00 -1.00000000e+00  9.25126959e-04 -2.67000734e-01
 -1.00000000e+00  1.06740275e-01 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  9.62603399e-01  2.24664722e-01  1.46801832e-01 -3.36755352e-01
 -4.75032671e-01 -1.00000000e+00 -1.00000000e+00 -7.50636814e-01
 -1.00000000e+00]


c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Fonction exploitable par le client

In [117]:
models = ['Arteon', 'Golf', 'Série 3', 'T-Roc', 'Zoe', 'Mazda 2', 'C-HR', 'Punto', 'A1', 'Mazda 3', 'C4', 'Puma', 'Arkana', '500', 'SpaceTourer', 'Tucson', 'Megane', 'Sandero', 'Focus', '5008', 'Karoq', 'Twingo', 'Leon', 'Golf Sportsvan', '508', 'Kadjar', 'Rio', 'Koleos', 'DS 7', 'Polo', 'Crossland X', 'Ibiza', 'Qashqai', 'C3', 'Grandland X', 'Clio', 'C5', '308', '1007', 'GS', 'Mazda 5', 'Passat', 'Vitara', 'A5', '207', 'Corsa', 'CX-5', 'MiTo', 'T-Cross', 'Mokka', 'Autre', 'Trafic Combi', 'DS 3', 'Grand Cherokee', '3008', 'Countryman', 'Matiz', 'Duster', 'C1', 'Talisman', 'TT', '2008', 'Q5', 'Autres', 'Kangoo', 'Captur', 'DS 4', 'C3 Aircross', 'Range Rover', 'C-MAX', 'MX-3', 'MX-5', 'Swift', 'C4 Picasso', 'XC60', 'Renegade', 'X1', '208', 'Grand C-MAX', 'Jumpy Fg', 'Jazz', 'Tiguan', 'CX-30', 'Rifter', 'Giulietta', 'Explorer', 'Touran', 'Micra', 'Scenic', 'Tipo', 'Giulia']
colors = ['Blanc','Gris','Noir','Vert','Bleu','Beige','Rouge','Marron','Argent','Jaune','Orange','Autre']
fuel = ['Diesel','Essence','Hybride','Electrique']
gearbox = ['Automatique','Manuelle']
brands = ['HONDA', 'LAND-ROVER', 'BMW', 'CHEVROLET', 'VOLKSWAGEN', 'CITROEN', 'MINI', 'HYUNDAI', 'MAZDA', 'DACIA', 'SEAT', 'FORD', 'LEXUS', 'AUDI', 'DS', 'ALFA ROMEO', 'FIAT', 'JEEP', 'PEUGEOT', 'SKODA', 'TOYOTA', 'NISSAN', 'OPEL', 'RENAULT', 'ABARTH', 'VOLVO', 'KIA', 'SUZUKI']

def normalise(array):
    mini = min(array)
    maxi = max(array)
    return((array-mini)/(maxi-mini))

def kNN(voitures1,note):
    X = np.zeros((len(note),7))
    Xpredi = np.zeros((len(voitures1),7))
    y = np.zeros(len(note))

    for i in range(len(note)):
        voiture,val = note[i]
        y[i] = val 
        X[i][0]=brands.index(voiture['brand'])
        X[i][1]=voiture['price']
        X[i][2]=models.index(voiture['model'])
        X[i][3]=voiture['km']
        X[i][4]=fuel.index(voiture['fuel'])
        X[i][5]=gearbox.index(voiture['gearbox'])
        X[i][6]=colors.index(voiture['color'])
        X[i] = normalise(X[i])
        
        
    
    param_grid = {'n_neighbors':[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25]}


# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
    clf = model_selection.GridSearchCV(
        KNeighborsRegressor(), # un classifieur kNN
        param_grid,     # hyperparamètres à tester
        cv=3,           # nombre de folds de validation croisée
     
)
# Optimiser ce classifieur sur le jeu d'entraînement
    clf.fit(X, y)
    nb_neighbors = clf.best_params_['n_neighbors']
    

    model = KNeighborsRegressor(nb_neighbors,weights='uniform')
    model.fit(X,y)

    for i in range(len(voitures1)):
         
        Xpredi[i][0]=brands.index(voitures1[i]['brand'])
        Xpredi[i][1]=voitures1[i]['price']
        Xpredi[i][2]=models.index(voitures1[i]['model'])
        Xpredi[i][3]=voitures1[i]['km']
        Xpredi[i][4]=fuel.index(voitures1[i]['fuel'])
        Xpredi[i][5]=gearbox.index(voitures1[i]['gearbox'])
        Xpredi[i][6]=colors.index(voitures1[i]['color'])
        Xpredi[i] = normalise(Xpredi[i])
        

    result = model.predict(Xpredi)
    
    

    return result

Un test de la fonction précédente

In [110]:
extract = like_2.loc[(like_2['swiped']=='t') & (like_2['user_id']== 1014),['car_id','value']]
X1 = pd.merge(extract,voitures)
y = X1['value']
X = X1.loc[:,['brand','model','km','price','fuel','gearbox','color']]
ind_train = X_train.index.tolist()
ind_test = X_test.index.tolist()
print(X)

       brand        model      km  price        fuel      gearbox   color
0      MAZDA         MX-3    4280  28990  Electrique  Automatique   Rouge
1    RENAULT       Scenic    6375  21900      Diesel     Manuelle   Beige
2      MAZDA      Mazda 2  173000   4490      Diesel     Manuelle  Orange
3     NISSAN      Qashqai  151560  10990      Diesel     Manuelle   Blanc
4    PEUGEOT          208    9345  21590     Essence  Automatique   Rouge
..       ...          ...     ...    ...         ...          ...     ...
219  PEUGEOT         2008   49637  19890     Essence     Manuelle   Blanc
220  CITROEN           C3  101787   7490     Essence     Manuelle    Gris
221  CITROEN  C3 Aircross   75481  13390     Essence     Manuelle    Gris
222     OPEL  Crossland X   58872  15480     Essence     Manuelle   Blanc
223  PEUGEOT         5008  175000   8990      Diesel  Automatique   Autre

[224 rows x 7 columns]


In [111]:

L1 = []
L2 = []
Y1 = []
for i in ind_train:
    x1 = dict(X.loc[i,:])
    y1 = X1.loc[i,'value']
    L1.append((x1,y1))

for i in ind_test:
    x1 = dict(X.loc[i,:])
    y1 = X1.loc[i,'value']
    L2.append(x1)
    Y1.append(y1)


print(len(L2))
print(len(X_test))
print(len(L1))
print(Y1)
    

        
    
        


45
45
179
[-1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1]


In [118]:
previ = kNN(L2,L1)
print(previ)


[-1.   -0.84 -0.36 -0.52 -0.44 -0.68 -0.28 -0.68 -0.84 -1.   -0.44 -0.52
 -0.52 -0.36 -0.52 -0.44 -0.76 -0.44 -1.   -0.44 -0.76 -0.68 -0.76 -0.52
 -0.6  -0.84 -0.28 -0.12 -0.76 -0.52 -1.   -0.44 -0.84 -0.76 -0.52 -1.
 -0.52 -0.36 -0.52 -0.84 -0.12 -1.   -0.52 -0.52 -0.2 ]
